# Pre-process data folder

Collect the information in "/Data" (or "/Data-snapshot" for the illustration) to generate the data to feed the evaluation. Collect the market information to describe the environment. Save the data for investments factor calculation in a pandas dataframe.

Presentation of the dataset then: 
 - Collection of the dataset for valuation of portfolio - (just the closing price of equities is required.)
 - Collection of the dataset for selection and factor construction - (price and other market information like equities debt, fx, commodities, equity index, etc)
 - Save the data that is collected into pansdas dataframes. 
 
COMMENT: "ISIN" defines the mnemonic to identify equities like isin="US4581401001" defines INTEL. Thus, isin and equity code refer to the same meaning.

Output in folder 'Data-processed':
* File of the time-series  = 'data_content.csv'
* File of the static definitions = 'data_risk.csv'
* File of the stock closing price = 'data_list.csv'

## 1. load the libraries

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import numpy as np
import pandas as pd
import os

##  2. Constants

Define Date as "YYYY-MM-DD" for end and start calculation - 10 year time-frame is considered consistent with years pre and post 2008 financial crisis)

NB: Post 2008 crisis is a single regime since that corresponds to the longest stock rally and therefore good strategies would be skewed towards getting exposure to risk, i.e. growth stocks.

In [2]:
DATE_MIN = "01/01/2004"  # Date minimum at which the evaluation starts. (Should contain the train/dev/text sets)
DATE_MAX = "01/01/2014"  # Date maximum at which the evaluation end. 

In [3]:
# Data description for asset classes (Equities, Indices, FX, Commodities, etc)
equities = "Equities Datastream Data Identifier_Cleaned.xls"
equities_indices = "Equities Indices Data Identifier_Cleaned.xls"
fx = "Exchange Rate Data Identifier_Cleaned.xls"
commodities = "Commodities Data Identifier_Cleaned.xls"
economic = "Economics Data Identifier_Cleaned.xls"
futures = "Futures Data Identifier_Cleaned.xls"
interest_rate = "Interest Rates Data Identifier_Cleaned.xls"
options = "Options Data Identifier_Cleaned.xls"

In [4]:
def print_codes(label):
    """
    Helper function to obtain the description of dataset per asset classes - ie
    options, equities, equities_indices, fx, commodities, economic, futures, interest_rate
    """
    name= pd.read_excel(os.path.join(os.getcwd(),"Data","Description","Data Type",label))
    name = name[['Name','Symbol','Type']]
    return name[name['Type']== 'Time Series'] , name[name['Type']== 'Static']

# Example of labels for stocks options
timeseries, static = print_codes(options)
# timeseries, static = print_codes(equities)
# timeseries, static = print_codes(equities_indices)
# timeseries, static = print_codes(fx)
# timeseries, static = print_codes(commodities)
# timeseries, static = print_codes(economic)
# timeseries, static = print_codes(futures)
# timeseries, static = print_codes(interest_rate)

In [5]:
# Illustration of data: time-series type of data (here: equities options)
timeseries.loc[:200, ['Name','Symbol']].set_index('Symbol').to_dict()

{'Name': {u'DZ': u'Delta',
  u'FV': u'Fair Value',
  u'GM': u'Gamma',
  u'I2': u'Implied Volatility - 2 Year Constant Maturity (Cont Series)',
  u'I3': u'Implied Volatility - 3 Year Constant Maturity (Cont Series)',
  u'I4': u'Implied Volatility - 4 Year Constant Maturity (Cont Series)',
  u'I5': u'Implied Volatility - 5 Year Constant Maturity (Cont Series)',
  u'MP': u'Price - Market',
  u'O1': u'Implied Volatility - 1 Month Constant Maturity (Cont Series)',
  u'O2': u'Implied Volatility - 2 Month Constant Maturity (Cont Series)',
  u'O3': u'Implied Volatility - 3 Month Constant Maturity (Cont Series)',
  u'O6': u'Implied Volatility - 6 Month Constant Maturity (Cont Series)',
  u'OC': u'Implied Volatility - Spot - Week Constant Maturity (Cont Series)',
  u'OD': u'Implied Volatility - 1 Week Constant Maturity (Cont Series)',
  u'OI': u'Open Interest',
  u'OM': u'At The Money Option Price (Cont Series)',
  u'ON': u'Implied Volatility - 9 Month Constant Maturity (Cont Series)',
  u'OS': 

In [6]:
# Illustration of data: Static type of data  - Typically description of financial instruments (here: equities options)
static.loc[0:100, ['Name','Symbol']].set_index('Symbol').to_dict()

{'Name': {u'MNEM': u'Mnemonic',
  u'NAME': u'Name',
  u'OEX': u'Exchange',
  u'OEXT': u'Expiry Style',
  u'OLOT': u'Lot Size (Single Series)',
  u'OPMOD': u'Option Model Calculation',
  u'OPTCODE': u'Exchange Ticker Symbol',
  u'OPTYPE': u'Option Underlying Instrument Type',
  u'OTYP': u'Option Type (Single Series)',
  u'OUI': u'Underlying Series Mnemonic',
  u'OUN': u'Underlying Series Name (Single Series)',
  u'OXCP': u'Exercise Price (Single Series)',
  u'OXPD': u'Expiry Date (Single Series)',
  u'PCUR': u'Price - Currency',
  u'TCYCLE': u'Trading Cycle',
  u'TIME': u'Time - Latest Value',
  u'TYPE': u'Type Of Instrument'}}

##  3. Selected data to collect

illustration fist of the financial information for equities and then illustration of the equities options information

###  3.1 Dictionnaires per asset class of the information collected

In [7]:
# Equities data - Time Series and Static
EQUITY_DEF = {u'P': u'Price (Adjusted - Default)',
              u'PH': u'Price - Intraday High',
              u'PL': u'Price - Intraday Low',
              u'PO': u'Price - Opening', 
              u'MVC':u'Market Value For Company'}
EQUITY_STATIC = {u'ISIN': u'Code - Isin',
                 u'NAME': u'Name'}
# Equities data - used for position computation - only closing price required
RISK_DEF = {u'P': u'Price (Adjusted - Default)'}


In [8]:
# Time-series data of financial information for equities
CORPORATE_DEF = {"WC08101":"QUICK RATIO",                                            
                 "WC08106":"CURRENT RATIO",  
                 "WC03051":"SHORT TERM DEBT & CURRENT PORTION OF LONG TERM DEBT",
                 "WC08311":"CASH FLOW/SALES",
                 "WC08316":"OPERATING PROFIT MARGIN",
                 "WC08221":"TOTAL DEBT % TOTAL CAPITAL & SHORT TERM DEBT",       
                 "WC08231":"TOTAL DEBT % COMMON EQUITY",
                 "WC18191":"EARNINGS BEFORE INTEREST AND TAXES (EBIT)",
                 "WC05376":"COMMON DIVIDENDS (CASH)",
                 "WC02999":"TOTAL ASSETS",
                 "WC01151":"DEPRECIATION, DEPLETION AND AMORTIZATION",
                 "WC03255":"TOTAL DEBT",
                 "WC02001":"CASH"}

In [9]:
# Continuous Options data for equities

# Comment: Continious options are instruments that represent the evolution of the different options for a unique
# derlying instruments. It avoids looping on all options quoted for that instrument.
CONTINUOUS_OPTION_DEF = {
    "MP":"Price - Market",
    "VI":"Implied Volatility - At The Money Interpolated (Cont Series)",
    "OI":"Open Interest",
    "VM":"Volume of Contracts Traded",
    "OD":"Implied Volatility - 1 Week Constant Maturity (Cont Series)",
    "O1":"Implied Volatility - 1 Month Constant Maturity (Cont Series)",
    "O3": "Implied Volatility - 3 Month Constant Maturity (Cont Series)",
    "O6": "Implied Volatility - 6 Month Constant Maturity (Cont Series)",
    "ON":"Implied Volatility - 9 Month Constant Maturity (Cont Series)",
    "OY":"Implied Volatility - 1 Year Constant Maturity (Cont Series)",
    "I3":"Implied Volatility - 3 Year Constant Maturity (Cont Series)"}

CONTINUOUS_OPTION_STATIC = {"MNEM": "Mnemonic",
                            "NAME":"Name",
                            "OPTYPE":"Option Underlying Instrument Type"}

In [10]:
# Interest Rate data - Time Series and Static
IR_DEF = {u'IR': u'Interest Rate'}
IR_STATIC = {u'MNEM': u'Mnemonic',u'NAME': u'Name',u'TYPE': u'Type Of Instrument'}

In [11]:
# Economic indictors data - Time Series and Static
ECO_DEF = {u'ES': u'Economic Series - Weekly'}
ECO_STATIC = {u'MNEM': u'Mnemonic',
              u'NAME': u'Name',
              u'TYPE': u'Type Of Instrument'}

In [12]:
# Commodities data - Time Series and Static
COMMO_DEF = {u'PO': u'Price - Opening',
             u'PH': u'Price - Intraday High',
             u'PL': u'Price - Intraday Low',
             u'P': u'Price (Adjusted - Default)'}
COMMO_STATIC = {u'MNEM': u'Mnemonic', u'NAME': u'Name'}

In [13]:
# Equity indices data - Time Series and Static
EQTYIND_DEF = {u'P': u'Price (Adjusted - Default)',
               u'PH': u'Price - Intraday High',
               u'PL': u'Price - Intraday Low',
               u'PO': u'Price - Opening'}
EQTYIND_STATIC = {u'ISIN': u'Code - Isin', u'NAME': u'Name'}

In [14]:
# Currencies data - Time Series and Static
FX_DEF = {u'ER': u'Exchange Rate - Middle'}
FX_STATIC = {u'MNEM': u'Mnemonic',
             u'NAME': u'Name',
             u'SICUR': u'Currency'}

##  4. Collect the data

Aggregate the different files of the financial instruments from DataStream (Reuters Financial) filtered with the information of interest through the previous dictionnaries (i.e. FX_DEF)

###  4.1 Time Series data

__Helper Functions__

In [15]:
 def process_df(path, filename, variable_isin):
    """
    Collect the variables in variable_isin for a isin in filename and output it into a pandas dataframe. 
    """
    test = pd.read_csv(os.path.join(path,filename), sep=";",
                       header=0,parse_dates=[0],infer_datetime_format=True,  encoding="utf-8")
    Col = list(test.columns)
    isin = Col[0]
    Col[0] = 'Date'
    test.columns = Col
    listcode = list(variable_isin)
    listcode.append('Date')
    test.drop(set(test.columns).difference(set(listcode)),axis=1, inplace=True) 
    test.columns = [isin + "-" + i for i in (test.columns)]
    test.set_index(isin + "-" +'Date', inplace=True)
    test.index.name = 'Date'
    return test.loc[(test.index > DATE_MIN) & (test.index < DATE_MAX)].dropna()

def collection(path_in, list_isin, variable_isin, cumul_df=None, first_df=True):
    """
    Process each of the codes (i.e. isin) to collect the data
    """
    source_path = os.path.join(os.getcwd(),*path_in)
    for position,filename in enumerate(list_isin):
        code_to_keep = variable_isin  # To avoid append "Date" at each computation
        if first_df:
            return process_df(source_path, filename, variable_isin)
        if position > 0:
            cumul_df = cumul_df.join(process_df(source_path, filename, variable_isin))
    return cumul_df

__Collect the data for time series__

In [16]:
# Collect the equities
def collect_ts(isin, variable):
    """
    Function to process a type of instruments and all the isin code given
    """
    df_collect = collection(['Data-snapshot','US Stocks'],isin, variable)
    return collection(['Data-snapshot','US Stocks'],isin, variable,df_collect, False)

# List of the equities - isin
list_isin = ['PA1436583006.txt',
             'US22160K1051.txt',
             'US4581401001.txt',
             'AN8068571086.txt']

collect_ts(list_isin, EQUITY_DEF.keys()).head()

/Users/pierrenowicki/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  from ipykernel import kernelapp as app


,PA1436583006-MVC,PA1436583006-P,PA1436583006-PH,PA1436583006-PL,PA1436583006-PO,US22160K1051-MVC,US22160K1051-P,US22160K1051-PH,US22160K1051-PL,US22160K1051-PO,US4581401001-MVC,US4581401001-P,US4581401001-PH,US4581401001-PL,US4581401001-PO,AN8068571086-MVC,AN8068571086-P,AN8068571086-PH,AN8068571086-PL,AN8068571086-PO
Date,,,,,,,,,,,,,,,,,,,,
2004-01-02,33610.74,39.82,40.25,39.750,39.91,16637.42,36.32,37.420,36.18,37.19,210068.9,32.16,32.600,32.03,32.36,32027.91,27.330,27.550,27.230,27.360
2004-01-05,34121.62,40.43,40.61,40.185,40.28,16559.55,36.15,36.610,35.86,36.44,214967.9,32.91,32.970,32.40,32.52,32525.97,27.755,27.780,26.990,27.100
2004-01-06,34297.64,40.75,40.80,40.300,40.43,16747.36,36.56,36.930,36.06,36.10,214902.7,32.90,33.000,32.62,32.92,32174.39,27.455,27.955,27.275,27.775
2004-01-07,34380.14,40.83,41.00,40.550,40.70,16985.55,37.08,37.175,36.52,36.58,222022.5,33.99,34.000,33.22,33.33,31377.51,26.775,27.330,26.645,27.330
2004-01-08,34754.78,41.24,41.30,40.850,40.87,17068.01,37.26,37.530,37.02,37.40,223655.6,34.24,34.349,33.90,34.17,31553.29,26.925,27.040,26.390,26.500


In [17]:
# Process the equity
list_isin = ['PA1436583006.txt',
             'US22160K1051.txt',
             'US4581401001.txt',
             'AN8068571086.txt']
variable_isin = EQUITY_DEF.keys()

collect_eq = collection(['Data-snapshot','US Stocks'],
           list_isin, variable_isin)
collect_eq = collection(['Data-snapshot','US Stocks'],
           list_isin, variable_isin,collect_eq, False)
collect_eq.head()

/Users/pierrenowicki/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2878: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,PA1436583006-MVC,PA1436583006-P,PA1436583006-PH,PA1436583006-PL,PA1436583006-PO,US22160K1051-MVC,US22160K1051-P,US22160K1051-PH,US22160K1051-PL,US22160K1051-PO,US4581401001-MVC,US4581401001-P,US4581401001-PH,US4581401001-PL,US4581401001-PO,AN8068571086-MVC,AN8068571086-P,AN8068571086-PH,AN8068571086-PL,AN8068571086-PO
Date,,,,,,,,,,,,,,,,,,,,
2004-01-02,33610.74,39.82,40.25,39.750,39.91,16637.42,36.32,37.420,36.18,37.19,210068.9,32.16,32.600,32.03,32.36,32027.91,27.330,27.550,27.230,27.360
2004-01-05,34121.62,40.43,40.61,40.185,40.28,16559.55,36.15,36.610,35.86,36.44,214967.9,32.91,32.970,32.40,32.52,32525.97,27.755,27.780,26.990,27.100
2004-01-06,34297.64,40.75,40.80,40.300,40.43,16747.36,36.56,36.930,36.06,36.10,214902.7,32.90,33.000,32.62,32.92,32174.39,27.455,27.955,27.275,27.775
2004-01-07,34380.14,40.83,41.00,40.550,40.70,16985.55,37.08,37.175,36.52,36.58,222022.5,33.99,34.000,33.22,33.33,31377.51,26.775,27.330,26.645,27.330
2004-01-08,34754.78,41.24,41.30,40.850,40.87,17068.01,37.26,37.530,37.02,37.40,223655.6,34.24,34.349,33.90,34.17,31553.29,26.925,27.040,26.390,26.500


In [18]:
# Process the equity for risk position calculus - P&L
list_isin = ['PA1436583006.txt',
             'US22160K1051.txt',
             'US4581401001.txt',
             'AN8068571086.txt']

variable_isin = RISK_DEF.keys()

collect_risk = collection(['Data-snapshot','US Stocks'],
           list_isin, RISK_DEF.keys())
collect_risk = collection(['Data-snapshot','US Stocks'],
           list_isin, RISK_DEF.keys(),collect_risk, False)
collect_risk.head()

,PA1436583006-P,US22160K1051-P,US4581401001-P,AN8068571086-P
Date,,,,
2004-01-02,39.82,36.32,32.16,27.330
2004-01-05,40.43,36.15,32.91,27.755
2004-01-06,40.75,36.56,32.90,27.455
2004-01-07,40.83,37.08,33.99,26.775
2004-01-08,41.24,37.26,34.24,26.925


In [19]:
# Process the equity for corporate finance
list_isin = ['PA1436583006.txt',
             'US22160K1051.txt',
             'US4581401001.txt',
             'AN8068571086.txt']
variable_isin = CORPORATE_DEF.keys()

collect_corp = collection(['Data-snapshot','US Stocks'],
           list_isin, CORPORATE_DEF.keys())
collect_corp = collection(['Data-snapshot','US Stocks'],
           list_isin, CORPORATE_DEF.keys(),collect_corp, False)
collect_corp.head()

,PA1436583006-WC01151,PA1436583006-WC02001,PA1436583006-WC02999,PA1436583006-WC03051,PA1436583006-WC03255,PA1436583006-WC05376,PA1436583006-WC08101,PA1436583006-WC08106,PA1436583006-WC08221,PA1436583006-WC08231,...,AN8068571086-WC03051,AN8068571086-WC03255,AN8068571086-WC05376,AN8068571086-WC08101,AN8068571086-WC08106,AN8068571086-WC08221,AN8068571086-WC08231,AN8068571086-WC08311,AN8068571086-WC08316,AN8068571086-WC18191
Date,,,,,,,,,,,,,,,,,,,,,
2004-01-05,812000.0,660000.0,27636000.0,1662000.0,7953000.0,400000.0,0.21,0.34,33.54,50.46,...,716000.0,4660052.0,441219.0,1.2,1.5,41.63,76.18,21.9,14.19,1442000.0
2004-01-12,812000.0,660000.0,27636000.0,1662000.0,7953000.0,400000.0,0.21,0.34,33.54,50.46,...,716000.0,4660052.0,441219.0,1.2,1.5,41.63,76.18,21.9,14.19,1442000.0
2004-01-19,812000.0,660000.0,27636000.0,1662000.0,7953000.0,400000.0,0.21,0.34,33.54,50.46,...,716000.0,4660052.0,441219.0,1.2,1.5,41.63,76.18,21.9,14.19,1442000.0
2004-01-26,812000.0,660000.0,27636000.0,1662000.0,7953000.0,400000.0,0.21,0.34,33.54,50.46,...,716000.0,4660052.0,441219.0,1.2,1.5,41.63,76.18,21.9,14.19,1442000.0
2004-02-02,812000.0,660000.0,27636000.0,1662000.0,7953000.0,400000.0,0.21,0.34,33.54,50.46,...,716000.0,4660052.0,441219.0,1.2,1.5,41.63,76.18,21.9,14.19,1442000.0


In [20]:
# Process the equities indicies
list_isin = ['NASA100.txt',
             'NASCOMP.txt',
             'S&PCOMP.txt',
             'CBOEVIX.txt']
variable_isin = EQTYIND_DEF.keys()

collect_eqind = collection(['Data-snapshot','Index'],
           list_isin, EQTYIND_DEF.keys())
collect_eqind = collection(['Data-snapshot','Index'],
           list_isin, EQTYIND_DEF.keys(),collect_eqind, False)
collect_eqind.head()

,NASA100-PH,NASA100-PL,NASA100-PO,NASCOMP-PH,NASCOMP-PL,NASCOMP-PO,S&PCOMP-PH,S&PCOMP-PL,S&PCOMP-PO,CBOEVIX-PH,CBOEVIX-PL,CBOEVIX-PO
Date,,,,,,,,,,,,
2004-06-14,1471.19,1451.78,1470.62,1987.83,1963.48,1987.75,1136.47,1122.16,1136.47,16.36,15.54,15.54
2004-06-15,1487.83,1470.37,1470.37,2006.58,1982.41,1982.41,1137.36,1125.29,1126.55,15.35,14.72,15.34
2004-06-16,1483.60,1474.17,1478.72,2002.07,1990.57,1997.10,1135.28,1130.55,1132.01,15.22,14.48,14.83
2004-06-17,1473.95,1458.68,1473.94,1993.93,1976.25,1993.69,1133.56,1126.88,1133.56,15.58,15.00,15.06
2004-06-18,1481.36,1455.67,1457.88,2000.41,1973.91,1977.77,1139.08,1129.82,1132.05,15.25,14.51,14.74


In [21]:
# Process the Commodities
list_isin = ['GOLDBLN.txt',
             'OILBREN.txt',
             'OILWTXI.txt',
             'SILVUSL.txt']
variable_isin = COMMO_DEF.keys()

collect_como = collection(['Data-snapshot','Commodities'],
           list_isin, COMMO_DEF.keys())
collect_como = collection(['Data-snapshot','Commodities'],
           list_isin, COMMO_DEF.keys(),collect_como, False)
collect_como.head()

,GOLDBLN-P,GOLDBLN-PH,GOLDBLN-PL,GOLDBLN-PO,OILBREN-P,OILBREN-PH,OILBREN-PL,OILWTXI-P,OILWTXI-PH,OILWTXI-PL,SILVUSL-P
Date,,,,,,,,,,,
2004-01-02,415.50,416.65,414.60,416.05,29.67,29.67,29.67,32.29,32.29,32.29,5.985
2004-01-05,421.75,424.55,417.25,418.00,31.24,31.24,31.24,33.76,33.76,33.76,5.975
2004-01-06,422.25,431.00,421.15,424.25,31.14,31.14,31.14,33.62,33.62,33.62,6.280
2004-01-07,423.25,424.85,420.25,421.75,31.17,31.17,31.17,33.57,33.57,33.57,6.215
2004-01-08,422.25,424.85,417.25,419.75,31.19,31.19,31.19,34.08,34.08,34.08,6.200


In [22]:
# Process the Exchange Rates - FX
list_isin = ['JAPAYE$.txt',
             'USDOLSF.txt']
variable_isin = FX_DEF.keys()

collect_fx = collection(['Data-snapshot','FX'],
           list_isin, FX_DEF.keys())
collect_fx = collection(['Data-snapshot','FX'],
           list_isin, FX_DEF.keys(),collect_fx, False)
collect_fx.head()

,JAPAYE$-ER,USDOLSF-ER
Date,,
2004-01-02,107.020,NaN
2004-01-05,106.135,NaN
2004-01-06,106.270,NaN
2004-01-07,106.160,NaN
2004-01-08,106.165,NaN


In [23]:
# Process the Options Continuous
list_isin = ['SPXC.SERIESC.txt',
             'SPXC.SERIESP.txt',
             'VIXC.SERIESC.txt',
             'VIXC.SERIESP.txt']
variable_isin = CONTINUOUS_OPTION_DEF.keys()

collect_cont_op = collection(['Data-snapshot','Option Index Continuous'],
           list_isin, CONTINUOUS_OPTION_DEF.keys())
collect_cont_op = collection(['Data-snapshot','Option Index Continuous'],
           list_isin, CONTINUOUS_OPTION_DEF.keys(),collect_cont_op, False)
collect_cont_op.head()

,SPXC.SERIESC-I3,SPXC.SERIESC-O1,SPXC.SERIESC-O3,SPXC.SERIESC-O6,SPXC.SERIESC-OI,SPXC.SERIESC-ON,SPXC.SERIESC-OY,SPXC.SERIESC-VI,SPXC.SERIESC-VM,SPXC.SERIESP-I3,...,VIXC.SERIESC-VI,VIXC.SERIESC-VM,VIXC.SERIESP-O1,VIXC.SERIESP-O3,VIXC.SERIESP-O6,VIXC.SERIESP-OI,VIXC.SERIESP-ON,VIXC.SERIESP-OY,VIXC.SERIESP-VI,VIXC.SERIESP-VM
Date,,,,,,,,,,,,,,,,,,,,,
2008-06-18,0.2380,0.2134,0.2157,0.2177,4948198.0,0.2197,0.2196,0.21301,359821.0,0.2385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-19,0.2378,0.2048,0.2058,0.2091,3060315.0,0.2162,0.2146,0.20559,396296.0,0.2370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-20,0.2418,0.2200,0.2150,0.2187,3182287.0,0.2182,0.2187,0.21879,256337.0,0.2372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-23,0.2421,0.2170,0.2163,0.2184,3222473.0,0.2186,0.2204,0.21153,187398.0,0.2347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-24,0.2316,0.2169,0.2199,0.2174,3391221.0,0.2246,0.1531,0.21630,248667.0,0.2369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# COMMENT: Economic data is split betweeen  Economic data and Economic Poll data
# Process Economic data
list_isin = ['USNAPMNO.txt', 'USPMNBC.Q.txt']
variable_isin = ECO_DEF.keys()
collect_eco = collection(['Data-snapshot','Economic'],
           list_isin, ECO_DEF.keys())
collect_eco = collection(['Data-snapshot','Economic'],
           list_isin, ECO_DEF.keys(),collect_eco, False)
collect_eco.head()

,USNAPMNO-ES,USPMNBC.Q-ES
Date,,
2004-01-15,70.6,NaN
2004-02-15,66.5,NaN
2004-03-15,64.6,NaN
2004-04-15,67.1,NaN
2004-05-15,64.5,NaN


In [25]:
# Process Economic Poll 
list_isin = ['USBSINV.B.txt',
             'USCAPUS.R.txt',
             'USCAPUTLQ.txt',
             'USCNFBUSQ.txt',
             'USCNFCONQ.txt',
             'USCRDCONB.txt',
            'USCSHPM%E.txt', 'USPENONFO.txt', 'USUMINM1R.txt']
variable_isin = ECO_DEF.keys()
collect_poll = collection(['Data-snapshot','Economic Poll'],
           list_isin, ECO_DEF.keys())
collect_poll = collection(['Data-snapshot','Economic Poll'],
           list_isin, ECO_DEF.keys(),collect_poll, False)
collect_poll.head()

,USBSINV.B-ES,USCAPUS.R-ES,USCAPUTLQ-ES,USCNFBUSQ-ES,USCNFCONQ-ES,USCRDCONB-ES,USCSHPM%E-ES,USPENONFO-ES,USUMINM1R-ES
Date,,,,,,,,,
2004-01-30,1148846,NaN,NaN,NaN,NaN,2088.24304,NaN,NaN,NaN
2004-02-27,1158974,NaN,NaN,NaN,NaN,2095.34036,NaN,NaN,NaN
2004-03-31,1168294,NaN,NaN,NaN,NaN,2109.72032,NaN,NaN,NaN
2004-04-30,1176015,NaN,NaN,NaN,NaN,2112.71615,NaN,NaN,NaN
2004-05-31,1183315,NaN,NaN,NaN,NaN,2120.83915,NaN,NaN,NaN


In [26]:
# Process Interest Rates
list_isin = ['ECSWF1Y.txt']
variable_isin = IR_DEF.keys()

collect_ir = collection(['Data-snapshot','Interest Rates'],
           list_isin, IR_DEF.keys())
collect_ir = collection(['Data-snapshot','Interest Rates'],
           list_isin, IR_DEF.keys(),collect_ir, False)
collect_ir.head()

,ECSWF1Y-IR
Date,
2004-01-02,0.5781
2004-01-05,0.5938
2004-01-06,0.5469
2004-01-07,0.5781
2004-01-08,0.5156


###  4.2 Static data

This is just to form a pandas dataframe with all mnemonics collected and their description

__Helper Functions__

In [27]:
# Process Static 
def process_s_df(path, filename, variable_isin):
    """
    Collect the variables for an isin. Save in a pandas dataframe. 
    path: folder name
    filename: file name where the isin information is located
    variable_isin: list of the variables for the isin to collect.
    """
    test = pd.read_csv(os.path.join(path,filename), sep=";",
                       header=0,infer_datetime_format=True,  encoding="utf-8")
    Col = list(test.columns)
    listcode = list(variable_isin)
    test.drop(set(test.columns).difference(set(listcode)),axis=1, inplace=True) 
    return test

def collection_s(path_in, list_isin, variable_isin, cumul_df=None, first_df=True):
    """
    Process the whole list of the isin for a type of financial instruments (equites, fx etc. )
    """
    source_path = os.path.join(os.getcwd(),*path_in)
    for position,filename in enumerate(list_isin):
        code_to_keep = variable_isin  # To avoid append "Date" at each computation
        if first_df:
            return process_s_df(source_path, filename, variable_isin)
        if position > 0:
            #print process_s_df(source_path, filename, variable_isin)
            cumul_df = pd.concat([cumul_df, process_s_df(source_path, filename, variable_isin)],axis=0)
    return cumul_df

__Collect__

In [28]:
# Process the equity Static information. 
list_isin = ['PA1436583006.txt',
             'US22160K1051.txt',
             'US4581401001.txt',
             'AN8068571086.txt']
variable_isin = EQUITY_STATIC.keys()

collect_s_eq = collection_s(['Data-snapshot','US Stocks Static'],
           list_isin, EQUITY_STATIC.keys())
collect_s_eq = collection_s(['Data-snapshot','US Stocks Static'],
           list_isin, EQUITY_STATIC.keys(),collect_s_eq, False)
# Rename ISIN into MENM for presentation purpose
collect_s_eq.columns = ['MNEM','NAME']
collect_s_eq['TYPE'] = 'Equity'
collect_s_eq.head()

,MNEM,NAME,TYPE
0,PA1436583006,CARNIVAL,Equity
0,US22160K1051,COSTCO WHOLESALE,Equity
0,US4581401001,INTEL,Equity
0,AN8068571086,SCHLUMBERGER,Equity


In [29]:
# Process the indicies - No information has been downloaded
# PUT THE DESCRIPTION FLAT and CONCATENATE
index_dict = {'CBOEVIX':'CHICAGO BOARD OF EXCHANGE VIX INDEX',
              'NASA100':'NASDAQ 100',
              'NASCOMP':'NASDAQ COMPOSITE',
              'S&PCOMP':'S&P 500 COMPOSITE'}

collect_s_index = pd.DataFrame.from_dict(index_dict,orient='index')
collect_s_index.index.name = 'MNEM'
collect_s_index.columns = ['NAME']
collect_s_index['TYPE'] = 'Index'
collect_s_index.reset_index(inplace=True)

In [30]:
# Process the Commodities
list_isin = ['GOLDBLN.txt',
             'OILBREN.txt',
             'OILWTXI.txt',
             'SILVUSL.txt']
variable_isin = COMMO_STATIC.keys()

collect_s_como = collection_s(['Data-snapshot','Commodities Static'],
           list_isin, COMMO_STATIC.keys())
collect_s_como = collection_s(['Data-snapshot','Commodities Static'],
           list_isin, COMMO_STATIC.keys(),collect_s_como, False)

collect_s_como['TYPE'] = 'Commodities'
collect_s_como.head()

,MNEM,NAME,TYPE
0,GOLDBLN,Gold Bullion LBM U$/Troy Ounce,Commodities
0,OILBREN,Crude Oil-Brent Cur. Month FOB U$/BBL,Commodities
0,OILWTXI,Crude Oil-WTI Near Month FOB U$/BBL,Commodities
0,SILVUSL,LBMA Silver Price USD/Troy Oz,Commodities


In [31]:
# Process the Exchange Rates
list_isin = ['JAPAYE$.txt',
             'USDOLSF.txt']
variable_isin = FX_STATIC.keys()

collect_s_fx = collection_s(['Data-snapshot','FX Static'],
           list_isin, FX_STATIC.keys())
collect_s_fx = collection_s(['Data-snapshot','FX Static'],
           list_isin, FX_STATIC.keys(),collect_s_fx, False)
collect_s_fx['TYPE'] = 'Currency'
collect_s_fx.head()

,MNEM,NAME,SICUR,TYPE
0,JAPAYE$,JAPANESE YEN TO US $ (WMR),U$,Currency
0,USDOLSF,US $ TO CHF (WMR),SF,Currency


In [32]:
# Process Economic
list_isin = ['USNAPMNO.txt','USBSINV.B.txt',
             'USCAPUS.R.txt',
             'USCAPUTLQ.txt',
             'USCNFBUSQ.txt',
             'USCNFCONQ.txt',
             'USCRDCONB.txt','USCSHPM%E.txt', 'USPENONFO.txt', 'USUMINM1R.txt']
variable_isin = ECO_STATIC.keys()
collect_s_eco = collection_s(['Data-snapshot','Economic Static'],
           list_isin, ECO_STATIC.keys())
collect_s_eco = collection_s(['Data-snapshot','Economic Static'],
           list_isin, ECO_STATIC.keys(),collect_s_eco, False)
collect_s_eco['TYPE'] = 'Economic'
collect_s_eco.head()

,MNEM,NAME,TYPE
0,USNAPMNO,US ISM MANUFACTURERS SURVEY: NEW ORDERS INDEX ...,Economic
0,USBSINV.B,US BUSINESS INVENTORIES (MFG & TRADE) CURA,Economic
0,USCAPUS.R,US REUTERS POLL: CAPACITY UTILIZATION (%MOM) -...,Economic
0,USCAPUTLQ,US CAPACITY UTILIZATION RATE - ALL INDUSTRY SADJ,Economic
0,USCNFBUSQ,US ISM PURCHASING MANAGERS INDEX (MFG SURVEY) ...,Economic


In [33]:
# Process Interest Rates
list_isin = ['ECSWF1Y.txt']
variable_isin = IR_STATIC.keys()
collect_s_ir = collection_s(['Data-snapshot','Interest Rates Static'],
           list_isin, IR_STATIC.keys())
collect_s_ir = collection_s(['Data-snapshot','Interest Rates Static'],
           list_isin, IR_STATIC.keys(),collect_s_ir, False)
collect_s_ir['TYPE'] = 'Interest Rate'
collect_s_ir.head()

,MNEM,NAME,TYPE
0,ECSWF1Y,SWISS FRANC 1Y DEPOSIT (FT/TR),Interest Rate


##  5. Output results

Save the pandas dataframes collected into 2 dataframes: time-series and static in "Data-processed" to build the 

In [34]:
# Concatenate and output the data in folder 'Data-processed'
df_time_series = pd.concat([collect_eq, collect_corp, collect_eqind,
           collect_como,
           collect_fx,
#            collect_op,
           collect_cont_op,
#            collect_fut,
           collect_eco,
           collect_poll,
           collect_ir], axis=1,sort=True)
df_time_series.to_csv(os.path.join(os.getcwd(),'Data-processed', 'data_content.csv'), encoding='utf-8')
# Output risk file in folder 'Data-processed'
collect_risk.to_csv(os.path.join(os.getcwd(),'Data-processed', 'data_risk.csv'), encoding='utf-8')
df_time_series.head()

,PA1436583006-MVC,PA1436583006-P,PA1436583006-PH,PA1436583006-PL,PA1436583006-PO,US22160K1051-MVC,US22160K1051-P,US22160K1051-PH,US22160K1051-PL,US22160K1051-PO,...,USBSINV.B-ES,USCAPUS.R-ES,USCAPUTLQ-ES,USCNFBUSQ-ES,USCNFCONQ-ES,USCRDCONB-ES,USCSHPM%E-ES,USPENONFO-ES,USUMINM1R-ES,ECSWF1Y-IR
Date,,,,,,,,,,,,,,,,,,,,,
2004-01-02,33610.74,39.82,40.25,39.750,39.91,16637.42,36.32,37.420,36.18,37.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5781
2004-01-05,34121.62,40.43,40.61,40.185,40.28,16559.55,36.15,36.610,35.86,36.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5938
2004-01-06,34297.64,40.75,40.80,40.300,40.43,16747.36,36.56,36.930,36.06,36.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5469
2004-01-07,34380.14,40.83,41.00,40.550,40.70,16985.55,37.08,37.175,36.52,36.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5781
2004-01-08,34754.78,41.24,41.30,40.850,40.87,17068.01,37.26,37.530,37.02,37.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5156


In [35]:
# Concatenate the description:
df_static = pd.concat([collect_s_eq,
           collect_s_index,
           collect_s_ir,
           collect_s_eco,
#           collect_s_fut[['MNEM','NAME','TYPE']],
           collect_s_como,
           collect_s_fx
          ],sort=True).set_index('MNEM')
df_static.to_csv(os.path.join(os.getcwd(),'Data-processed', 'data_list.csv'), encoding='utf-8')
df_static

,NAME,SICUR,TYPE
MNEM,,,
PA1436583006,CARNIVAL,NaN,Equity
US22160K1051,COSTCO WHOLESALE,NaN,Equity
US4581401001,INTEL,NaN,Equity
AN8068571086,SCHLUMBERGER,NaN,Equity
NASCOMP,NASDAQ COMPOSITE,NaN,Index
NASA100,NASDAQ 100,NaN,Index
S&PCOMP,S&P 500 COMPOSITE,NaN,Index
CBOEVIX,CHICAGO BOARD OF EXCHANGE VIX INDEX,NaN,Index
ECSWF1Y,SWISS FRANC 1Y DEPOSIT (FT/TR),NaN,Interest Rate
